# Compute OPR, Ranking point, and Win ratings

In [1]:
from __future__ import print_function
import pickle
#import time
import sys
sys.path.append('..')
import swagger_client as v3client
from swagger_client.rest import ApiException
from collections import Counter
from IPython.display import HTML

In [2]:


HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [104]:

year = 2020
events = []
#events = ['2020waahs', '2020wasno', '2020pncmp']

filename = 'matches_{}.pkl'.format(year)
teamname = 'teams_{}.pkl'.format(year)
matches = []
with open(filename, 'rb') as f:
    matches = pickle.load(f)


teams = []
with open(teamname, 'rb') as f:
    teams = pickle.load(f)
    
teamlookup = dict([(t.key,t.nickname) for t in teams])
    
print('Loaded {} events and {} teams'.format(len(matches['matches']),len(teams)))

Loaded 192 events and 3914 teams


In [105]:
if (len(events)>0):
    print("Using only events {}".format(','.join(events)))
non_empty = [k for k in matches['matches'].keys() if len(matches['matches'][k])>0]
data = [m for k in matches['matches'] for m in matches['matches'][k] if k in events or len(events)==0]
data = [m for m in data if m.winning_alliance!='']
#print(len(data))
red = [x for m in data for x in m.alliances.red.team_keys]
blue = [x for m in data for x in m.alliances.blue.team_keys]

Counter(red+blue)

# qualifying matches
qualifiers = [m for m in data if m.comp_level=='qm']
alliances = [' '.join(m.alliances.red.team_keys) for m in qualifiers]  + [' '.join(m.alliances.blue.team_keys) for m in qualifiers]
scores = [m.alliances.red.score for m in qualifiers] + [m.alliances.blue.score for m in qualifiers]
rp = [m.score_breakdown and m.score_breakdown['red']['rp'] or 0.0 for m in qualifiers] + [m.score_breakdown and m.score_breakdown['blue']['rp'] or 0.0 for m in qualifiers]
wins = [int(m.winning_alliance=='red') for m in qualifiers] + [int(m.winning_alliance=='blue') for m in qualifiers]

# finals (we'll ignore these for now)
finals = [m for m in data if m.comp_level!='qm']
alliances_finals = [' '.join(m.alliances.red.team_keys) for m in finals]  + [' '.join(m.alliances.blue.team_keys) for m in finals]
scores_finals = [m.alliances.red.score for m in finals] + [m.alliances.blue.score for m in finals]
#len(scores),len(scores_finals)  # should be 2x number of qualifiers
print('{} total matches'.format(len(data)))

29 total matches


In [106]:
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.pipeline import FeatureUnion
from sklearn.compose import ColumnTransformer
import numpy as np
#from scipy.sparse.linalg import spsolve

# two count vectorizers. This transforms the alliance lists into vector encodings
vectorizer = CountVectorizer(max_features=15000, min_df=1, max_df=1.0, stop_words=None)  

X = vectorizer.fit_transform(alliances)
y = np.column_stack([scores, rp, wins])

# we'll ignore for now
X_finals = vectorizer.transform(alliances_finals)
y_finals = scores_finals

#X[1,:].todense()
#np.shape(y)


In [107]:
from scipy.sparse.linalg import lsqr

oprating = lsqr(X,y[:,0])[0]
rprating = lsqr(X,y[:,1])[0]
winrating = lsqr(X,y[:,2])[0]
ratings_all = np.column_stack([oprating, rprating, winrating])
rating_lookup=dict(zip(sorted(vectorizer.vocabulary_,key=lambda x: vectorizer.vocabulary_[x]),ratings_all))
#print(rating_lookup['frc492'])

In [108]:
teamAggregates = {}

def addMatch(team, m):    
    if team not in teamAggregates:       
        # initialize an empty record for the team.
        teamAggregates[team]= {
            'name': teamlookup[team] if team in teamlookup else 'unknown',
            'totalMatches':0, 
            'rpTotal': 0,
            'totalPoints': 0,
            'winCount': 0
        }
            

    alliance = 'blue' if team in m.alliances.blue.team_keys else 'red'
    points = m.score_breakdown[alliance]
    summary = teamAggregates[team]
    # update all of the fields.
    summary['totalMatches']+=1    
    
    summary['rpTotal'] += points['rp']
    summary['totalPoints'] += points['totalPoints']
    summary['winCount'] += int(m.winning_alliance==alliance)

# Currently just using Week0 events.
for m in qualifiers:
    if m.score_breakdown is None:
        continue
    # only aggregate statistics for regional matches- skip the districts or we're cheating.
    #if m.event_key=='2020pncmp':
    #    continue
    for t in m.alliances.red.team_keys:
        addMatch(t,m)
    for t in m.alliances.blue.team_keys:
        addMatch(t,m)
        
# normalize the aggregates
skipped = 0
for t in teamAggregates:
    for k in teamAggregates[t]:
        if k=='totalMatches' or k=='name':
            continue
        teamAggregates[t][k]/=teamAggregates[t]['totalMatches']
    ratings = ['opr', 'rp_rating', 'win_rating']
    if t in rating_lookup:
        for i,r in enumerate(ratings):
            teamAggregates[t][r]=rating_lookup[t][i]        
    else:
        skipped+=1

print('Skipped {} teams with missing data'.format(skipped))

with open('teamStats_tmp_2019.pkl','wb') as f:
    pickle.dump(teamAggregates,f)

Skipped 0 teams with missing data


In [109]:
# Save to tsv for processing in excel.
with open('stats_{}.tsv'.format(year),'w',encoding='utf-8') as outStats:
    outStats.write('{}\n'.format('\t'.join(['team','name','opr','rp_rating','win_rating','rp','points','wins'])))
    for t in teamAggregates:
        k = teamAggregates[t]
        if 'opr' not in k:
            continue
        outStats.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(t,k['name'], k['opr'],k['rp_rating'],k['win_rating'], k['rpTotal'],k['totalPoints'],k['winCount']))

In [110]:
from pandas import DataFrame

df = DataFrame.from_dict(dict(filter(lambda x: 'opr' in x[1],teamAggregates.items())), orient='index') 
df.index.name = 'team'


In [111]:
"""
Copyright 2019, Marek Cermak
Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
"""

def init_datatable_mode():
    """Initialize DataTable mode for pandas DataFrame represenation."""
    import pandas as pd
    from IPython.core.display import display, Javascript

    # configure path to the datatables library using requireJS
    # that way the library will become globally available
    display(Javascript("""
        require.config({
            paths: {
                DT: '//cdn.datatables.net/1.10.19/js/jquery.dataTables.min',
            }
        });
        $('head').append('<link rel="stylesheet" type="text/css" href="//cdn.datatables.net/1.10.19/css/jquery.dataTables.min.css">');
    """))

    def _repr_datatable_(self):
        """Return DataTable representation of pandas DataFrame."""
        # classes for dataframe table (optional)
        classes = ['table', 'table-striped', 'table-bordered']

        # create table DOM
        script = (
            f'$(element).html(`{self.to_html(index=True, classes=classes)}`);\n'
        )

        # execute jQuery to turn table into DataTable
        script += """
            require(["DT"], function(DT) {
                $(document).ready( () => {
                    // Turn existing table into datatable
                    $(element).find("table.dataframe").DataTable();
                })
            });
        """

        return script

    pd.DataFrame._repr_javascript_ = _repr_datatable_

In [112]:
import numpy as np
import pandas as pd
init_datatable_mode()  # initialize [DataTables]
df

<IPython.core.display.Javascript object>

,name,totalMatches,rpTotal,totalPoints,winCount,opr,rp_rating,win_rating
team,,,,,,,,
frc1073,The Force Team,3,2.000000,20.666667,1.000000,16.768144,0.651367,0.325683
frc1153,The Revolution,4,1.500000,36.250000,0.750000,49.447234,0.965215,0.482608
frc151,Tough Techs,3,1.333333,9.666667,0.666667,48.839349,3.529162,1.764581
frc1519,Mechanical Mayhem,3,2.000000,34.333333,1.000000,-32.936642,-0.232706,-0.116353
frc157,AZTECHS,3,0.000000,17.000000,0.000000,-20.297568,-1.982411,-0.991206
frc1721,Tidal Force,3,0.666667,22.666667,0.333333,-60.316595,-3.542043,-1.771021
frc1729,Team Inconceivable!,3,0.000000,10.000000,0.000000,-38.893880,-2.047143,-1.023571
frc2079,4H ALARM Robotics,3,1.333333,21.666667,0.666667,-26.459200,0.407938,0.203969
frc2084,Robots by the C,3,0.666667,33.666667,0.333333,-18.305218,-1.465150,-0.732575
